In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import influence_plot

In [2]:
import pandas as pd

file_path = "Amazon Beauty Recommendation system.csv"  # Use the exact file name
df = pd.read_csv(file_path)

# Check if the file loaded correctly
print(df.head())
print(df.info())


           UserId  ProductId  Rating   Timestamp
0  A39HTATAQ9V7YF  205616461       5  1369699200
1  A3JM6GV9MNOF9X  558925278       3  1355443200
2  A1Z513UWSAAO0F  558925278       5  1404691200
3  A1WMRR494NWEWV  733001998       4  1382572800
4  A3IAAVS479H7M7  737104473       1  1274227200
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   UserId     1048575 non-null  object
 1   ProductId  1048575 non-null  object
 2   Rating     1048575 non-null  int64 
 3   Timestamp  1048575 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 32.0+ MB
None


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   UserId     1048575 non-null  object
 1   ProductId  1048575 non-null  object
 2   Rating     1048575 non-null  int64 
 3   Timestamp  1048575 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 32.0+ MB


In [4]:
df.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,205616461,5,1369699200
1,A3JM6GV9MNOF9X,558925278,3,1355443200
2,A1Z513UWSAAO0F,558925278,5,1404691200
3,A1WMRR494NWEWV,733001998,4,1382572800
4,A3IAAVS479H7M7,737104473,1,1274227200


In [5]:
df.describe()

,Rating,Timestamp
count,1.048575e+06,1.048575e+06
mean,4.176864e+00,1.345636e+09
std,1.306543e+00,5.567113e+07
min,1.000000e+00,9.087552e+08
25%,4.000000e+00,1.322438e+09
50%,5.000000e+00,1.362960e+09
75%,5.000000e+00,1.385856e+09
max,5.000000e+00,1.406074e+09


In [6]:
df.size

4194300

In [7]:
unique_users = df["UserId"].nunique()
unique_products = df["ProductId"].nunique()

print(f"Unique Users: {unique_users}")
print(f"Unique Products: {unique_products}")


Unique Users: 736653
Unique Products: 97987


In [8]:
user_counts = df["UserId"].value_counts()
product_counts = df["ProductId"].value_counts()

print("Top Repeated Users:\n", user_counts.head())
print("Top Repeated Products:\n", product_counts.head())


Top Repeated Users:
 UserId
A3M174IC0VXOS2    278
A281NPSIMI1C2R    245
A1RRMZKOMZ2M7J    200
A3KEZLJ59C1JVH    178
A2V5R832QCSOMX    173
Name: count, dtype: int64
Top Repeated Products:
 ProductId
B001MA0QY2    7533
B0009V1YR8    2869
B0000YUXI0    2143
B000ZMBSPE    2041
B003BQ6QXK    1918
Name: count, dtype: int64


In [9]:
import pandas as pd
from scipy.sparse import csr_matrix

# Load dataset
df = pd.read_csv('Amazon Beauty Recommendation system.csv')

# Convert categorical IDs to numerical indices (helps with memory efficiency)
user_map = {user: idx for idx, user in enumerate(df['UserId'].unique())}
product_map = {product: idx for idx, product in enumerate(df['ProductId'].unique())}

df['UserId'] = df['UserId'].map(user_map)
df['ProductId'] = df['ProductId'].map(product_map)

# Create sparse pivot table
sparse_pivot = csr_matrix(
    (df['Rating'], (df['UserId'], df['ProductId']))
)

print("Sparse matrix shape:", sparse_pivot.shape)


Sparse matrix shape: (736653, 97987)


In [10]:
import pandas as pd

# Load the dataset
file_path = "Amazon Beauty Recommendation system.csv"  # Adjust if renamed
df = pd.read_csv(file_path)

# Reduce dataset size by filtering popular products and active users
popular_products = df['ProductId'].value_counts().nlargest(5000).index  # Top 5000 products
active_users = df['UserId'].value_counts().nlargest(5000).index  # Top 5000 users

df_filtered = df[df['ProductId'].isin(popular_products) & df['UserId'].isin(active_users)]

# Generate pivot table
pivot_table = df_filtered.pivot_table(index='UserId', columns='ProductId', values='Rating', aggfunc='mean', fill_value=0)

# Display pivot table sample
print(pivot_table.head())


ProductId              9746427962  9790778309  9790790961  B00004TMFE  \
UserId                                                                  
A02155413BVL8D0G7X6DN         0.0         0.0         0.0         0.0   
A03364251DGXSGA9PSR99         0.0         0.0         0.0         0.0   
A100WO06OQR8BQ                0.0         0.0         0.0         0.0   
A1010QRG4BH51B                0.0         0.0         0.0         0.0   
A101NJYKW882Y0                0.0         0.0         0.0         0.0   

ProductId              B00004TUBL  B00004TUBV  B00004U9UY  B00004U9V2  \
UserId                                                                  
A02155413BVL8D0G7X6DN         0.0         0.0         0.0         0.0   
A03364251DGXSGA9PSR99         0.0         0.0         0.0         0.0   
A100WO06OQR8BQ                0.0         0.0         0.0         0.0   
A1010QRG4BH51B                0.0         0.0         0.0         0.0   
A101NJYKW882Y0                0.0         0.0     

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
# Compute User-User Similarity Matrix
user_similarity = cosine_similarity(pivot_table)
# Convert to DataFrame for readability
user_similarity_df = pd.DataFrame(user_similarity, index=pivot_table.index, columns=pivot_table.index)
# Display sample similarity matrix
print(user_similarity_df.head())

UserId                 A02155413BVL8D0G7X6DN  A03364251DGXSGA9PSR99  \
UserId                                                                
A02155413BVL8D0G7X6DN                    1.0                    0.0   
A03364251DGXSGA9PSR99                    0.0                    1.0   
A100WO06OQR8BQ                           0.0                    0.0   
A1010QRG4BH51B                           0.0                    0.0   
A101NJYKW882Y0                           0.0                    0.0   

UserId                 A100WO06OQR8BQ  A1010QRG4BH51B  A101NJYKW882Y0  \
UserId                                                                  
A02155413BVL8D0G7X6DN             0.0             0.0             0.0   
A03364251DGXSGA9PSR99             0.0             0.0             0.0   
A100WO06OQR8BQ                    1.0             0.0             0.0   
A1010QRG4BH51B                    0.0             1.0             0.0   
A101NJYKW882Y0                    0.0             0.0           

In [12]:
# Compute Item-Item Similarity Matrix
item_similarity = cosine_similarity(pivot_table.T)
# Convert to DataFrame for readability
item_similarity_df = pd.DataFrame(item_similarity, index=pivot_table.columns, columns=pivot_table.columns)
# Display sample item similarity matrix
print(item_similarity_df.head())

ProductId   9746427962  9790778309  9790790961  B00004TMFE  B00004TUBL  \
ProductId                                                                
9746427962         1.0         0.0         0.0         0.0         0.0   
9790778309         0.0         1.0         0.0         0.0         0.0   
9790790961         0.0         0.0         1.0         0.0         0.0   
B00004TMFE         0.0         0.0         0.0         1.0         0.0   
B00004TUBL         0.0         0.0         0.0         0.0         1.0   

ProductId   B00004TUBV  B00004U9UY  B00004U9V2  B000050B6U  B000050B75  ...  \
ProductId                                                               ...   
9746427962         0.0         0.0         0.0         0.0         0.0  ...   
9790778309         0.0         0.0         0.0         0.0         0.0  ...   
9790790961         0.0         0.0         0.0         0.0         0.0  ...   
B00004TMFE         0.0         0.0         0.0         0.0         0.0  ...   
B00004T

In [13]:
def get_similar_users(user_id, top_n=5):
    return user_similarity_df[user_id].sort_values(ascending=False).iloc[1:top_n+1]
# Find top 5 similar users to a given user
user_id = pivot_table.index[0]  # Replace with an actual user ID if needed
print(f"Top similar users for User {user_id}:\n", get_similar_users(user_id))

Top similar users for User A02155413BVL8D0G7X6DN:
 UserId
A14EGTIUFVIIS4    0.265837
A25VUNGYCUFEN0    0.263846
A3MPZJ19V4E34H    0.235639
A3VPHOHZVHW9Y0    0.235639
A9MTN0D260WAD     0.225161
Name: A02155413BVL8D0G7X6DN, dtype: float64


In [14]:
def get_similar_items(product_id, top_n=5):
    return item_similarity_df[product_id].sort_values(ascending=False).iloc[1:top_n+1]
#Find top 5 similar products to a given product
product_id = pivot_table.columns[0]  # Replace with an actual product ID if needed
print(f"Top similar items for Product {product_id}:\n", get_similar_items(product_id))

Top similar items for Product 9746427962:
 ProductId
B00306CISI    0.370625
B0002Z8SE8    0.338062
B001G3VA22    0.292603
B001KYO08S    0.273434
B00397SNMS    0.263523
Name: 9746427962, dtype: float64


In [15]:
def recommend_products_for_user(user_id, top_n=5000):
    similar_users = get_similar_users(user_id, top_n)
    user_ratings = pivot_table.loc[similar_users.index].mean()
    return user_ratings.sort_values(ascending=False).head(top_n)
#Get recommended products for a user
print(f"Recommended products for User {user_id}:\n", recommend_products_for_user(user_id))

Recommended products for User A02155413BVL8D0G7X6DN:
 ProductId
B000142FVW    0.125152
B000ZMBSPE    0.118864
B00150LT40    0.084787
B000TKH6G2    0.072819
B001OMI93S    0.072617
                ...   
B001E96M94    0.000203
B000GOVYE6    0.000203
B000C1VXI0    0.000203
B001AHUP34    0.000203
B002J6YIC4    0.000000
Length: 4627, dtype: float64


In [16]:
def get_similar_items(product_id, top_n=5000):
    return item_similarity_df[product_id].sort_values(ascending=False).iloc[1:top_n+1]
#Find top 5 similar products to a given product
product_id = pivot_table.columns[0]  # Replace with an actual product ID if needed
print(f"Top similar items for Product {product_id}:\n", get_similar_items(product_id))

Top similar items for Product 9746427962:
 ProductId
B00306CISI    0.370625
B0002Z8SE8    0.338062
B001G3VA22    0.292603
B001KYO08S    0.273434
B00397SNMS    0.263523
                ...   
B000NBBN1G    0.000000
B000NB5AWY    0.000000
B000NB3VOS    0.000000
B000NB3VNO    0.000000
B003H897AQ    0.000000
Name: 9746427962, Length: 4626, dtype: float64


In [17]:
def recommend_products_for_user(user_id):
    similar_users = get_similar_users(user_id)
    user_ratings = pivot_table.loc[similar_users.index].mean()
    return user_ratings.sort_values(ascending=False)
#Get recommended products for a user
print(f"Recommended products for User {user_id}:\n", recommend_products_for_user(user_id))

Recommended products for User A02155413BVL8D0G7X6DN:
 ProductId
B002787ACO    2.0
B001A6N5GE    1.0
B003FO70Z6    1.0
B002LE8OVA    1.0
B000TW2JQC    1.0
             ... 
B000NBIRII    0.0
B000NBBN1G    0.0
B000NB5AWY    0.0
B000NB3VOS    0.0
B003H897AQ    0.0
Length: 4627, dtype: float64


In [18]:
def recommend_similar_items(product_id):
    return get_similar_items(product_id)
#Get recommended similar items for a product
print(f"Recommended similar items for Product {product_id}:\n", recommend_similar_items(product_id))

Recommended similar items for Product 9746427962:
 ProductId
B00306CISI    0.370625
B0002Z8SE8    0.338062
B001G3VA22    0.292603
B001KYO08S    0.273434
B00397SNMS    0.263523
                ...   
B000NBBN1G    0.000000
B000NB5AWY    0.000000
B000NB3VOS    0.000000
B000NB3VNO    0.000000
B003H897AQ    0.000000
Name: 9746427962, Length: 4626, dtype: float64
